In [ ]:
# df["Article_Content"] = df["Article_Content"].fillna("")

# # Label posts
# tqdm.pandas()
# df["is_relevant"] = df["Article_Content"].progress_apply(is_relevant_llm)
# print("Relevant posts:", df["is_relevant"].sum())
# texts = df["Article_Content"].tolist()
# batched_results = ner_pipe(texts, batch_size=16)
# df["company_name"] = [extract_company(r) for r in batched_results]

# print("Company names extracted.")
# df["techniques"] = df["Article_Content"].progress_apply(extract_techniques)
# print("Techniques extracted.")
# print(df)

# # save the dataframe
# df.to_csv("reportparse/reddit_db/data/reddit_greenwashing_posts_enhanced.csv", index=False)

# handler = RedditChromaHandler()

# # Store all posts (or filter as needed)
# for _, row in tqdm(df.iterrows(), total=len(df)):
#     handler.store_post(
#         post_id=str(row["Post_id"]),
#         text=row["Article_Content"],
#         metadata={
#             "url": row["Article_Url"],
#             "is_relevant": row["is_relevant"],
#             "company": ", ".join(row["company_name"]) if isinstance(row["company_name"], list) else str(row["company_name"]),
#             "techniques": ", ".join(row["techniques"]) if isinstance(row["techniques"], list) else str(row["techniques"]),
#         },
#     )
# print("Posts stored.")

In [1]:
import pandas as pd
from tqdm import tqdm
from relevance_classifier import is_relevant_llm
from ner_extractor import extract_company
from technique_tagger import extract_techniques
from reddit_chroma_handler import RedditChromaHandler
from transformers import pipeline

ner_pipe = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")

# Load data
df = pd.read_csv("data/checkpoint3.csv")


/home/geoka/miniconda3/envs/rocm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [2]:
# Eda
print(df.head())
print(df.columns)

   Unnamed: 0  Type Post_id  \
0          14  Post  8xs4vu   
1          15  Post  8xsb69   
2          16  Post  98lw7v   
3          17  Post  9uq7ll   
4          18  Post  9zxj8o   

                                               Title         Author  \
0         We don't wait for the future. We build it.       wewewawa   
1  Is Wireless Phone Giant Using Hawaii To Greenw...       wewewawa   
2  Making people feel good about throwing things ...  shasha_neequa   
3  Do you know about Greenwashing and Big Organic...       mjan2010   
4  Go to r/greenwash for a more active subreddit ...         ckeeks   

             Timestamp Text  \
0  2018-07-10 18:51:17  NaN   
1  2018-07-10 19:10:41  NaN   
2  2018-08-19 17:27:19  NaN   
3  2018-11-06 17:13:23  NaN   
4  2018-11-24 10:36:44  NaN   

                                         Content_URL  Score  Total_comments  \
0                      https://www.humanability.com/      3               0   
1  https://www.civilbeat.org/2018/07/is-w

In [3]:
df.head()

,Unnamed: 0,Type,Post_id,Title,Author,Timestamp,Text,Content_URL,Score,Total_comments,Post_URL,label,Type of content,Year,Content,text_for_emb,text_for_emb_length
0,14,Post,8xs4vu,We don't wait for the future. We build it.,wewewawa,2018-07-10 18:51:17,NaN,https://www.humanability.com/,3,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Invalid URL or Unreachable,2018,NaN,Title: We don't wait for the future. We build ...,17
1,15,Post,8xsb69,Is Wireless Phone Giant Using Hawaii To Greenw...,wewewawa,2018-07-10 19:10:41,NaN,https://www.civilbeat.org/2018/07/is-wireless-...,2,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,Verizon Communications Inc. is publicizing Haw...,Title: Is Wireless Phone Giant Using Hawaii To...,541
2,16,Post,98lw7v,Making people feel good about throwing things ...,shasha_neequa,2018-08-19 17:27:19,NaN,https://www.reddit.com/r/mildlyinteresting/com...,2,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,We value your privacy\n\nReddit and its partne...,Title: Making people feel good about throwing ...,101
3,17,Post,9uq7ll,Do you know about Greenwashing and Big Organic...,mjan2010,2018-11-06 17:13:23,NaN,https://www.naturalnews.com/2018-11-05-organic...,1,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,"Organic JUNK food? You can waste your money, w...",Title: Do you know about Greenwashing and Big ...,740
4,18,Post,9zxj8o,Go to r/greenwash for a more active subreddit ...,ckeeks,2018-11-24 10:36:44,NaN,https://www.reddit.com/r/Greenwash/,5,1,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,We value your privacy\n\nReddit and its partne...,Title: Go to r/greenwash for a more active sub...,101


In [5]:
# drop unnamed: 0 collumn
df = df.drop(columns=["Unnamed: 0"])

In [6]:
# if the column "Content" AND && "Text"  is empty, drop the row

df = df.dropna(subset=["Content", "Text"], how="all")

In [7]:
df.head()

,Type,Post_id,Title,Author,Timestamp,Text,Content_URL,Score,Total_comments,Post_URL,label,Type of content,Year,Content,text_for_emb,text_for_emb_length
1,Post,8xsb69,Is Wireless Phone Giant Using Hawaii To Greenw...,wewewawa,2018-07-10 19:10:41,NaN,https://www.civilbeat.org/2018/07/is-wireless-...,2,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,Verizon Communications Inc. is publicizing Haw...,Title: Is Wireless Phone Giant Using Hawaii To...,541
2,Post,98lw7v,Making people feel good about throwing things ...,shasha_neequa,2018-08-19 17:27:19,NaN,https://www.reddit.com/r/mildlyinteresting/com...,2,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,We value your privacy\n\nReddit and its partne...,Title: Making people feel good about throwing ...,101
3,Post,9uq7ll,Do you know about Greenwashing and Big Organic...,mjan2010,2018-11-06 17:13:23,NaN,https://www.naturalnews.com/2018-11-05-organic...,1,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,"Organic JUNK food? You can waste your money, w...",Title: Do you know about Greenwashing and Big ...,740
4,Post,9zxj8o,Go to r/greenwash for a more active subreddit ...,ckeeks,2018-11-24 10:36:44,NaN,https://www.reddit.com/r/Greenwash/,5,1,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,We value your privacy\n\nReddit and its partne...,Title: Go to r/greenwash for a more active sub...,101
7,Post,cbnido,The Greening of The West Leaves Other Countrie...,StopFossilFuels,2019-07-10 22:32:29,NaN,http://achnews.org/2019/06/27/the-greening-of-...,1,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2019,Use multmetric.com to build a height compariso...,Title: The Greening of The West Leaves Other C...,300


In [8]:
# rename all columns to lowercase
df.columns = df.columns.str.lower()

In [9]:
df.head()

,type,post_id,title,author,timestamp,text,content_url,score,total_comments,post_url,label,type of content,year,content,text_for_emb,text_for_emb_length
1,Post,8xsb69,Is Wireless Phone Giant Using Hawaii To Greenw...,wewewawa,2018-07-10 19:10:41,NaN,https://www.civilbeat.org/2018/07/is-wireless-...,2,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,Verizon Communications Inc. is publicizing Haw...,Title: Is Wireless Phone Giant Using Hawaii To...,541
2,Post,98lw7v,Making people feel good about throwing things ...,shasha_neequa,2018-08-19 17:27:19,NaN,https://www.reddit.com/r/mildlyinteresting/com...,2,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,We value your privacy\n\nReddit and its partne...,Title: Making people feel good about throwing ...,101
3,Post,9uq7ll,Do you know about Greenwashing and Big Organic...,mjan2010,2018-11-06 17:13:23,NaN,https://www.naturalnews.com/2018-11-05-organic...,1,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,"Organic JUNK food? You can waste your money, w...",Title: Do you know about Greenwashing and Big ...,740
4,Post,9zxj8o,Go to r/greenwash for a more active subreddit ...,ckeeks,2018-11-24 10:36:44,NaN,https://www.reddit.com/r/Greenwash/,5,1,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,We value your privacy\n\nReddit and its partne...,Title: Go to r/greenwash for a more active sub...,101
7,Post,cbnido,The Greening of The West Leaves Other Countrie...,StopFossilFuels,2019-07-10 22:32:29,NaN,http://achnews.org/2019/06/27/the-greening-of-...,1,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2019,Use multmetric.com to build a height compariso...,Title: The Greening of The West Leaves Other C...,300


In [10]:
# rename text_for_emb collumnt to text_summary and text_for_emb_length to text_summary_length
df = df.rename(columns={"text_for_emb": "text_summary", "text_for_emb_length": "text_summary_length"})

In [11]:
df.head()

,type,post_id,title,author,timestamp,text,content_url,score,total_comments,post_url,label,type of content,year,content,text_summary,text_summary_length
1,Post,8xsb69,Is Wireless Phone Giant Using Hawaii To Greenw...,wewewawa,2018-07-10 19:10:41,NaN,https://www.civilbeat.org/2018/07/is-wireless-...,2,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,Verizon Communications Inc. is publicizing Haw...,Title: Is Wireless Phone Giant Using Hawaii To...,541
2,Post,98lw7v,Making people feel good about throwing things ...,shasha_neequa,2018-08-19 17:27:19,NaN,https://www.reddit.com/r/mildlyinteresting/com...,2,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,We value your privacy\n\nReddit and its partne...,Title: Making people feel good about throwing ...,101
3,Post,9uq7ll,Do you know about Greenwashing and Big Organic...,mjan2010,2018-11-06 17:13:23,NaN,https://www.naturalnews.com/2018-11-05-organic...,1,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,"Organic JUNK food? You can waste your money, w...",Title: Do you know about Greenwashing and Big ...,740
4,Post,9zxj8o,Go to r/greenwash for a more active subreddit ...,ckeeks,2018-11-24 10:36:44,NaN,https://www.reddit.com/r/Greenwash/,5,1,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,We value your privacy\n\nReddit and its partne...,Title: Go to r/greenwash for a more active sub...,101
7,Post,cbnido,The Greening of The West Leaves Other Countrie...,StopFossilFuels,2019-07-10 22:32:29,NaN,http://achnews.org/2019/06/27/the-greening-of-...,1,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2019,Use multmetric.com to build a height compariso...,Title: The Greening of The West Leaves Other C...,300


In [12]:
df["text"] = df["text"].fillna("")
df["content"] = df["content"].fillna("")


texts = df["text_summary"].tolist()
batched_results = ner_pipe(texts, batch_size=16)
df["company_name"] = [extract_company(r) for r in batched_results]

/home/geoka/miniconda3/envs/rocm/lib/python3.10/site-packages/torch/nn/modules/linear.py:125: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:310.)
  return F.linear(input, self.weight, self.bias)


In [13]:
df.head()

,type,post_id,title,author,timestamp,text,content_url,score,total_comments,post_url,label,type of content,year,content,text_summary,text_summary_length,company_name
1,Post,8xsb69,Is Wireless Phone Giant Using Hawaii To Greenw...,wewewawa,2018-07-10 19:10:41,,https://www.civilbeat.org/2018/07/is-wireless-...,2,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,Verizon Communications Inc. is publicizing Haw...,Title: Is Wireless Phone Giant Using Hawaii To...,541,"[Wireless Phone Giant, Verizon Communications ..."
2,Post,98lw7v,Making people feel good about throwing things ...,shasha_neequa,2018-08-19 17:27:19,,https://www.reddit.com/r/mildlyinteresting/com...,2,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,We value your privacy\n\nReddit and its partne...,Title: Making people feel good about throwing ...,101,"[Reddit, Reddit, Red, Reddit, Not]"
3,Post,9uq7ll,Do you know about Greenwashing and Big Organic...,mjan2010,2018-11-06 17:13:23,,https://www.naturalnews.com/2018-11-05-organic...,1,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,"Organic JUNK food? You can waste your money, w...",Title: Do you know about Greenwashing and Big ...,740,"[USDA, Food, Can, USDA]"
4,Post,9zxj8o,Go to r/greenwash for a more active subreddit ...,ckeeks,2018-11-24 10:36:44,,https://www.reddit.com/r/Greenwash/,5,1,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,We value your privacy\n\nReddit and its partne...,Title: Go to r/greenwash for a more active sub...,101,"[Reddit, Reddit, ##dit, Reddit, Not]"
7,Post,cbnido,The Greening of The West Leaves Other Countrie...,StopFossilFuels,2019-07-10 22:32:29,,http://achnews.org/2019/06/27/the-greening-of-...,1,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2019,Use multmetric.com to build a height compariso...,Title: The Greening of The West Leaves Other C...,300,"[CASINOR, ##L, OCryptoCanada, Doulike, com, Do..."


In [14]:
# make company_name collumn list contain only unique values

df["company_name"] = df["company_name"].apply(lambda x: list(set(x)) if isinstance(x, list) else x)

In [15]:
df.head()

,type,post_id,title,author,timestamp,text,content_url,score,total_comments,post_url,label,type of content,year,content,text_summary,text_summary_length,company_name
1,Post,8xsb69,Is Wireless Phone Giant Using Hawaii To Greenw...,wewewawa,2018-07-10 19:10:41,,https://www.civilbeat.org/2018/07/is-wireless-...,2,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,Verizon Communications Inc. is publicizing Haw...,Title: Is Wireless Phone Giant Using Hawaii To...,541,"[Verizon Communications Inc, O, Big, T - Mobil..."
2,Post,98lw7v,Making people feel good about throwing things ...,shasha_neequa,2018-08-19 17:27:19,,https://www.reddit.com/r/mildlyinteresting/com...,2,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,We value your privacy\n\nReddit and its partne...,Title: Making people feel good about throwing ...,101,"[Red, Reddit, Not]"
3,Post,9uq7ll,Do you know about Greenwashing and Big Organic...,mjan2010,2018-11-06 17:13:23,,https://www.naturalnews.com/2018-11-05-organic...,1,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,"Organic JUNK food? You can waste your money, w...",Title: Do you know about Greenwashing and Big ...,740,"[USDA, Food, Can]"
4,Post,9zxj8o,Go to r/greenwash for a more active subreddit ...,ckeeks,2018-11-24 10:36:44,,https://www.reddit.com/r/Greenwash/,5,1,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2018,We value your privacy\n\nReddit and its partne...,Title: Go to r/greenwash for a more active sub...,101,"[Reddit, Not, ##dit]"
7,Post,cbnido,The Greening of The West Leaves Other Countrie...,StopFossilFuels,2019-07-10 22:32:29,,http://achnews.org/2019/06/27/the-greening-of-...,1,0,https://www.reddit.com/r/greenwashing/comments...,Greenwashing,Text Page,2019,Use multmetric.com to build a height compariso...,Title: The Greening of The West Leaves Other C...,300,"[com, Stigan Media, CASINOR, Drip Casino, Do, ..."


In [18]:
# print unique values of type of content
print(df["type of content"].unique())

['Text Page' 'No content found' 'Image' 'Unknown Content' 'YouTube Video'
 'Video' 'Invalid URL or Unreachable']


In [16]:
# save df as checkpoint4.csv
df.to_csv("data/checkpoint4.csv", index=False)

In [ ]:
# Store all posts (or filter as needed)
import pandas as pd
from tqdm import tqdm
from relevance_classifier import is_relevant_llm
from ner_extractor import extract_company
from technique_tagger import extract_techniques
from reddit_chroma_handler import RedditChromaHandler
from transformers import pipeline

df = pd.read_csv("data/checkpoint4.csv")

handler = RedditChromaHandler()

for _, row in tqdm(df.iterrows(), total=len(df)):
    handler.store_post(
        post_id=str(row["post_id"]),
        text=row["text_summary"],
        metadata={
            "content_url": row["content_url"],
            "post_url": row["post_url"],
            "subreddit": row["label"],
            "company": ", ".join(row["company_name"]) if isinstance(row["company_name"], list) else str(row["company_name"]),
        },
    )
print("Posts stored.")

100%|██████████| 5180/5180 [06:52<00:00, 12.56it/s]

Posts stored.
